**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Outside we must switch on ingress

`minikube addons enable ingress`

In [1]:
kubectl get pods -A | grep --color=always -E '$|ingress'

NAMESPACE       NAME                                        READY   STATUS      RESTARTS      AGE
ingress-nginx   ingress-nginx-admission-create--1-hh8g4     0/1     Completed   0             47s
ingress-nginx   ingress-nginx-admission-patch--1-jvcx5      0/1     Completed   0             47s
ingress-nginx   ingress-nginx-controller-69bdbc4d57-zcn4l   1/1     Running     0             47s
kube-system     coredns-78fcd69978-n2jxz                    1/1     Running     0             93m
kube-system     etcd-minikube                               1/1     Running     0             93m
kube-system     kube-apiserver-minikube                     1/1     Running     0             94m
kube-system     kube-controller-manager-minikube            1/1     Running     0             94m
kube-system     kube-proxy-rcwrr                            1/1     Running     0             93m
kube-system     kube-scheduler-minikube                     1/1     Running     0             93m
kube-system     stor

In [ ]:
kubectl delete deployment nginx-strict
kubectl delete svc nginx-strict


In [2]:
kubectl get svc -A

NAMESPACE       NAME                                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                      AGE
default         kubernetes                           ClusterIP   10.96.0.1       <none>        443/TCP                      95m
ingress-nginx   ingress-nginx-controller             NodePort    10.99.217.223   <none>        80:31838/TCP,443:32604/TCP   2m15s
ingress-nginx   ingress-nginx-controller-admission   ClusterIP   10.108.145.96   <none>        443/TCP                      2m15s
kube-system     kube-dns                             ClusterIP   10.96.0.10      <none>        53/UDP,53/TCP,9153/TCP       95m


In [3]:
kubectl create -f - <<EOF
apiVersion: apps/v1 
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
        ports:
        - containerPort: 80
EOF

deployment.apps/nginx-deployment created


In [4]:
kubectl get all

NAME                                    READY   STATUS    RESTARTS   AGE
pod/nginx-deployment-7848d4b86f-btn6m   1/1     Running   0          15s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   96m

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-deployment   1/1     1            1           15s

NAME                                          DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-deployment-7848d4b86f   1         1         1       15s


In [5]:
kubectl create -f - <<EOF
kind: Service
apiVersion: v1
metadata:
  name: "nginx"
spec:
  selector:
    app: "nginx"
  ports:
    - protocol: "TCP"
      port: 80
      targetPort: 80
  type: LoadBalancer
EOF

service/nginx created


In [7]:
kubectl get svc

NAME         TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)        AGE
kubernetes   ClusterIP      10.96.0.1        <none>           443/TCP        98m
nginx        LoadBalancer   10.102.252.206   10.102.252.206   80:30405/TCP   46s


In [8]:
kubectl describe service nginx

Name:                     nginx
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=nginx
Type:                     LoadBalancer
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.102.252.206
IPs:                      10.102.252.206
LoadBalancer Ingress:     10.102.252.206
Port:                     <unset>  80/TCP
TargetPort:               80/TCP
NodePort:                 <unset>  30405/TCP
Endpoints:                172.17.0.3:80
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [9]:
kubectl get pods --all-namespaces

NAMESPACE       NAME                                        READY   STATUS      RESTARTS      AGE
default         nginx-deployment-7848d4b86f-btn6m           1/1     Running     0             2m7s
ingress-nginx   ingress-nginx-admission-create--1-hh8g4     0/1     Completed   0             5m34s
ingress-nginx   ingress-nginx-admission-patch--1-jvcx5      0/1     Completed   0             5m34s
ingress-nginx   ingress-nginx-controller-69bdbc4d57-zcn4l   1/1     Running     0             5m34s
kube-system     coredns-78fcd69978-n2jxz                    1/1     Running     0             98m
kube-system     etcd-minikube                               1/1     Running     0             98m
kube-system     kube-apiserver-minikube                     1/1     Running     0             98m
kube-system     kube-controller-manager-minikube            1/1     Running     0             98m
kube-system     kube-proxy-rcwrr                            1/1     Running     0             98m
kube-system  

In [10]:
kubectl get svc --all-namespaces

NAMESPACE       NAME                                 TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)                      AGE
default         kubernetes                           ClusterIP      10.96.0.1        <none>           443/TCP                      98m
default         nginx                                LoadBalancer   10.102.252.206   10.102.252.206   80:30405/TCP                 104s
ingress-nginx   ingress-nginx-controller             NodePort       10.99.217.223    <none>           80:31838/TCP,443:32604/TCP   5m45s
ingress-nginx   ingress-nginx-controller-admission   ClusterIP      10.108.145.96    <none>           443/TCP                      5m45s
kube-system     kube-dns                             ClusterIP      10.96.0.10       <none>           53/UDP,53/TCP,9153/TCP       98m


In [11]:
kubectl get configmap -n ingress-nginx

NAME                        DATA   AGE
ingress-controller-leader   0      5m51s
ingress-nginx-controller    1      6m4s
kube-root-ca.crt            1      6m4s
tcp-services                0      6m4s
udp-services                0      6m4s


In [12]:
kubectl get configmap -n  ingress-nginx  ingress-controller-leader -o yaml

apiVersion: v1
kind: ConfigMap
metadata:
  annotations:
    control-plane.alpha.kubernetes.io/leader: '{"holderIdentity":"ingress-nginx-controller-69bdbc4d57-zcn4l","leaseDurationSeconds":30,"acquireTime":"2022-03-29T10:52:00Z","renewTime":"2022-03-29T10:58:01Z","leaderTransitions":0}'
  creationTimestamp: "2022-03-29T10:52:00Z"
  name: ingress-controller-leader
  namespace: ingress-nginx
  resourceVersion: "4798"
  uid: 206c2dbe-e1fc-4b65-a15a-067a5471af9a


In [13]:
kubectl get configmap -n  ingress-nginx ingress-nginx-controller -o yaml

apiVersion: v1
data:
  hsts: "false"
kind: ConfigMap
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","data":{"hsts":"false"},"kind":"ConfigMap","metadata":{"annotations":{},"labels":{"app.kubernetes.io/component":"controller","app.kubernetes.io/instance":"ingress-nginx","app.kubernetes.io/name":"ingress-nginx"},"name":"ingress-nginx-controller","namespace":"ingress-nginx"}}
  creationTimestamp: "2022-03-29T10:51:47Z"
  labels:
    app.kubernetes.io/component: controller
    app.kubernetes.io/instance: ingress-nginx
    app.kubernetes.io/name: ingress-nginx
  name: ingress-nginx-controller
  namespace: ingress-nginx
  resourceVersion: "4363"
  uid: c8a6879f-a0e1-4891-ba17-b75c9a1d99a8
